# Compare catchment datasources

In this notebook, we compare the different catchment datasources:
- federal agencies -> where available
- MERIT Hydro
- Hydrosheds
- Basis-EZGs (by Pia)

The catchment boundaries provided by the federal agencies are used as some sort of benchmark. As we only have these catchment for a fraction of catchments and shareability is questionable, we have to use one of the other datasources for the CAMELS-DE dataset.  
So we compare the similarity / overlap of the MERIT Hydro, Hydrosheds and Basis-EZGs with the "official" (but incomplete) catchment boundaries provided by the federal agencies.

We calculate 4 metrics in total:
1. `overlap ratio`, this mean Intersect / Union
2. `distance between centroids`
3. `difference between area of shapes`
4. `difference to reported area`, for most of the catchments we have an area reported in the metadata

In [1]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import plotly.graph_objects as go

from camelsp import get_metadata, Station

## Helper functions

In [2]:
def get_area_of_gdf(gdf: gpd.GeoDataFrame) -> float:
    """
    Calculate the area of a catchment in km2.  
    The crs of the GeoDataFrame is transformed to an crs in meters (epsg:6933) 
    before calculating the area.

    Parameters:
    -------
    gdf : gpd.GeoDataFrame
        GeoDataFrame of the catchment.

    Returns:
    -------
    area: float
        Area of the catchment in km2.
    """
    if gdf is None:
        return None
    else:
        # transform to crs in meters
        gdf = gdf.to_crs(epsg=6933)
        
        # calculate area
        area = gdf.area.sum() / 1e6 # in km2

        return area
    

## 1.) Overlap Ratio

In [3]:
def calculate_overlap_ratio_two_catchments(gdf_base: gpd.GeoDataFrame, gdf_compare: gpd.GeoDataFrame) -> float:
    """
    Calculate the ratio of overlap between two catchments.  
    This is calculated as the area of the intersection between the two catchments 
    divided by the union of both catchments.
    Returns None if gdf_compare is None.

    Parameters:
    -------
    gdf_base : gpd.GeoDataFrame
        GeoDataFrame of the base catchment.
    gdf_compare : gpd.GeoDataFrame
        GeoDataFrame of the catchment to compare.

    Returns:
    -------
    overlap_ratio: float
        Ratio of overlapping area to union of both areas between the two catchments.

    """
    if gdf_compare is None:
        return None
    
    # Calculate the intersection between the two catchments
    intersection = gdf_base.intersection(gdf_compare)

    # Calculate the union of the two catchments
    union = gdf_base.union(gdf_compare)
    
    # Calculate the area of the intersection and union
    intersection_area = get_area_of_gdf(intersection)
    union_area = get_area_of_gdf(union)
    
    # Calculate the overlap ratio
    overlap_ratio = intersection_area / union_area # TODO: normalize between 0 and 1, too big and too small values would cancel each other out in the mean
    
    return overlap_ratio


## 2.) Distance between centroids

possible to normalize this?

In [4]:
def calculate_distance_between_centroids(gdf_base: gpd.GeoDataFrame, gdf_compare: gpd.GeoDataFrame):
    """
    Calculate the distance in m between the centroids of two catchments.  
    Returns None if gdf_compare is None.

    Parameters:
    -------
    gdf_base : gpd.GeoDataFrame
        GeoDataFrame of the base catchment.
    gdf_compare : gpd.GeoDataFrame
        GeoDataFrame of the catchment to compare.

    Returns:
    -------
    distance: float
        Distance between the centroids of the two catchments in m.

    """
    if gdf_compare is None:
        return None
    
    # transform to crs in meters
    gdf_base = gdf_base.to_crs(epsg=6933)
    gdf_compare = gdf_compare.to_crs(epsg=6933)
    
    # Calculate the centroid of the two catchments
    centroid_base = gdf_base.centroid
    centroid_compare = gdf_compare.centroid

    # Calculate the distance between the two centroids in m
    distance = centroid_base.distance(centroid_compare).values[0] # TODO: normalization
    
    return distance

## 3.) Difference between area of shapes

In [5]:
def calculate_area_difference(gdf_base: gpd.GeoDataFrame, gdf_compare: gpd.GeoDataFrame):
    """
    Calculate the difference in area in m2 between two catchments.  
    Returns None if gdf_compare is None.

    Parameters:
    -------
    gdf_base : gpd.GeoDataFrame
        GeoDataFrame of the base catchment.
    gdf_compare : gpd.GeoDataFrame
        GeoDataFrame of the catchment to compare.

    Returns:
    -------
    ratio_area_difference: float
        Relative difference in area between the two catchments.

    """
    if gdf_compare is None:
        return None
    
    # Calculate the area of the two catchments
    area_base = get_area_of_gdf(gdf_base)
    area_compare = get_area_of_gdf(gdf_compare)

    # Calculate the absolute difference in area between the two catchments in km2
    ratio_area_difference = abs(area_base - area_compare) / area_base # TODO: normalization
    
    return ratio_area_difference

## 4.) Difference to reported area

In [12]:
def calculate_relative_difference_to_area_reported(area_reported: float, gdf_compare: gpd.GeoDataFrame):
    """
    Calculate the relative difference in area between a catchment and the reported area.  
    Returns None if gdf_compare is None.

    Parameters:
    -------
    area_reported : float
        Reported area of the catchment in km2.
    gdf_compare : gpd.GeoDataFrame
        GeoDataFrame of the catchment to compare.

    Returns:
    -------
    ratio_area_difference: float
        Ratio of difference in area between reported area and area of the catchment.  
        Equals 0 when the reported area is equal to the area of the catchment.

    """
    if gdf_compare is None:
        return None
    if area_reported is None or area_reported == 0:
        return None
    
    # Calculate the area of the catchment
    area_compare = get_area_of_gdf(gdf_compare)

    # Calculate the relative difference in area between the reported area and the area of the catchment
    ratio_area_difference = abs(area_reported - area_compare) / area_reported

    return ratio_area_difference


## Execute everything in one cell

In [29]:
# get metadata
metadata = get_metadata()
#metadata = metadata[metadata["flag_q_more_than_10_years"]]

# get all camels ids
camels_ids = metadata["camels_id"].values

# initialize dataframe with camels_ids as index
df_results = pd.DataFrame(index=camels_ids)

for camels_id in camels_ids:
    # initialize station
    s = Station(camels_id)

    # get catchments for all datasources, check if available
    gdf_federal_agency_ezg = s.get_catchment("federal_agency_ezg")
    gdf_merit_hydro = s.get_catchment("merit_hydro")
    gdf_hydrosheds = s.get_catchment("hydrosheds")
    gdf_basis_ezg = s.get_catchment("basis_ezg")

    # get reported area
    area_reported = s.metadata["area"].values[0]

    # fill with None if no catchment or areaa_reported is available
    if gdf_federal_agency_ezg is None:
        df_results.loc[s.camels_id, "overlap_fed_merit_hydro"] = None
        df_results.loc[s.camels_id, "overlap_fed_hydrosheds"] = None
        df_results.loc[s.camels_id, "overlap_fed_basis_ezg"] = None
        df_results.loc[s.camels_id, "centroid_distance_fed_merit_hydro"] = None
        df_results.loc[s.camels_id, "centroid_distance_fed_hydrosheds"] = None
        df_results.loc[s.camels_id, "centroid_distance_fed_basis_ezg"] = None
        df_results.loc[s.camels_id, "area_difference_fed_merit_hydro"] = None
        df_results.loc[s.camels_id, "area_difference_fed_hydrosheds"] = None
        df_results.loc[s.camels_id, "area_difference_fed_basis_ezg"] = None
    if area_reported is None:
        df_results.loc[s.camels_id, "area_difference_reported_fed_merit_hydro"] = None
        df_results.loc[s.camels_id, "area_difference_reported_fed_hydrosheds"] = None
        df_results.loc[s.camels_id, "area_difference_reported_fed_basis_ezg"] = None

    # calculate metrics
    if gdf_federal_agency_ezg is not None:
        # caluculate percentage of overlap to federal agencies
        overlap_merit_hydro = calculate_overlap_ratio_two_catchments(gdf_federal_agency_ezg, gdf_merit_hydro)
        overlap_hydrosheds = calculate_overlap_ratio_two_catchments(gdf_federal_agency_ezg, gdf_hydrosheds)
        # DEA10710: GeosError when calculating the intersection and union
        if camels_id == "DEA10710":
            overlap_basis_ezg = None
        else:
            overlap_basis_ezg = calculate_overlap_ratio_two_catchments(gdf_federal_agency_ezg, gdf_basis_ezg)

        # save results
        df_results.loc[s.camels_id, "overlap_fed_merit_hydro"] = overlap_merit_hydro
        df_results.loc[s.camels_id, "overlap_fed_hydrosheds"] = overlap_hydrosheds
        df_results.loc[s.camels_id, "overlap_fed_basis_ezg"] = overlap_basis_ezg
        df_results.loc[s.camels_id, "centroid_distance_fed_merit_hydro"] = calculate_distance_between_centroids(gdf_federal_agency_ezg, gdf_merit_hydro)
        df_results.loc[s.camels_id, "centroid_distance_fed_hydrosheds"] = calculate_distance_between_centroids(gdf_federal_agency_ezg, gdf_hydrosheds)
        df_results.loc[s.camels_id, "centroid_distance_fed_basis_ezg"] = calculate_distance_between_centroids(gdf_federal_agency_ezg, gdf_basis_ezg)
        df_results.loc[s.camels_id, "area_difference_fed_merit_hydro"] = calculate_area_difference(gdf_federal_agency_ezg, gdf_merit_hydro)
        df_results.loc[s.camels_id, "area_difference_fed_hydrosheds"] = calculate_area_difference(gdf_federal_agency_ezg, gdf_hydrosheds)
        df_results.loc[s.camels_id, "area_difference_fed_basis_ezg"] = calculate_area_difference(gdf_federal_agency_ezg, gdf_basis_ezg)
    df_results.loc[s.camels_id, "area_difference_reported_fed_merit_hydro"] = calculate_relative_difference_to_area_reported(area_reported, gdf_merit_hydro)
    df_results.loc[s.camels_id, "area_difference_reported_fed_hydrosheds"] = calculate_relative_difference_to_area_reported(area_reported, gdf_hydrosheds)
    df_results.loc[s.camels_id, "area_difference_reported_fed_basis_ezg"] = calculate_relative_difference_to_area_reported(area_reported, gdf_basis_ezg)

# make all columns float
df_results = df_results.astype(float)

## Inspect results

In [30]:
df_results.describe()

,overlap_fed_merit_hydro,overlap_fed_hydrosheds,overlap_fed_basis_ezg,centroid_distance_fed_merit_hydro,centroid_distance_fed_hydrosheds,centroid_distance_fed_basis_ezg,area_difference_fed_merit_hydro,area_difference_fed_hydrosheds,area_difference_fed_basis_ezg,area_difference_reported_fed_merit_hydro,area_difference_reported_fed_hydrosheds,area_difference_reported_fed_basis_ezg
count,1066.000000,1074.000000,1068.000000,1066.000000,1074.000000,1069.000000,1.066000e+03,1.074000e+03,1.069000e+03,2732.000000,2712.000000,2677.000000
mean,0.829064,0.492145,0.724619,2545.549048,9271.720427,12369.248948,5.570540e+03,6.004969e+03,8.250331e+00,44.464206,84.690896,11.068654
std,0.252810,0.321881,0.406609,12832.174207,22597.715495,33375.538610,1.766013e+05,1.853149e+05,1.384536e+02,966.949379,1334.657626,383.573792
min,0.000000,0.000000,0.000000,1.985461,11.305525,0.000083,2.010423e-05,1.617822e-04,1.068850e-10,0.000005,0.000066,0.000004
25%,0.843455,0.172813,0.561497,47.009419,1701.414861,40.918089,4.244023e-03,1.463674e-01,3.610016e-03,0.004730,0.118560,0.003055
50%,0.929499,0.530716,0.965661,174.833700,4238.170148,273.962364,1.582939e-02,6.737597e-01,3.218649e-02,0.021497,0.737038,0.022653
75%,0.965459,0.803105,0.995000,610.890735,7490.039861,2594.773328,7.744439e-02,3.829456e+00,3.191518e-01,0.113558,5.084203,0.225564
max,0.994541,0.986580,1.000000,215404.995425,252289.758079,307997.166674,5.764495e+06,6.066951e+06,3.924660e+03,33681.888437,43426.417610,18928.183440


### Difference to area_reported

In [32]:
df_results["area_difference_reported_fed_merit_hydro"][df_results["area_difference_reported_fed_merit_hydro"] > 0.05]

DEG10040       0.244245
DEG10100       0.063115
DEG10110       0.082459
DEG10190      42.551945
DEG10250       0.051986
               ...     
DE112340       3.382116
DE112460       0.740880
DE112470    2547.921936
DE112490       0.063634
DE112500       0.158258
Name: area_difference_reported_fed_merit_hydro, Length: 1000, dtype: float64

### Histogram

In [9]:
# plot histogram with plotly go
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_results["overlap_fed_merit_hydro"], name="MERIT Hydro", xbins=dict(start=0, end=1, size=0.05)))
fig.add_trace(go.Histogram(x=df_results["overlap_fed_hydrosheds"], name="HydroSHEDS", xbins=dict(start=0, end=1, size=0.05)))
fig.add_trace(go.Histogram(x=df_results["overlap_fed_basis_ezg"], name="BASIS-EZG", xbins=dict(start=0, end=1, size=0.05)))
fig.update_layout(barmode='overlay', title="Overlap of Federal Agency Catchments with other Datasets")
fig.update_traces(opacity=0.6)
fig.update_xaxes(title_text="Overlap [%]")
fig.update_yaxes(title_text="Count")
fig.show()


In [11]:
# find stations with overlap < 0.1
df_results[df_results["overlap_fed_merit_hydro"] < 0.1]

,overlap_fed_merit_hydro,overlap_fed_hydrosheds,overlap_fed_basis_ezg,centroid_distance_fed_merit_hydro,centroid_distance_fed_hydrosheds,centroid_distance_fed_basis_ezg,area_difference_fed_merit_hydro,area_difference_fed_hydrosheds,area_difference_fed_basis_ezg,area_difference_reported_fed_merit_hydro,area_difference_reported_fed_hydrosheds,area_difference_reported_fed_basis_ezg
DEF10580,5.796524e-04,8.433604e-02,5.542194e-01,14346.330182,6793.483150,540.344638,3.434835e+01,9.930871e+00,8.043373e-01,214.559833,66.348947,10.952089
DEF14070,1.149048e-02,1.059785e-02,9.179876e-01,19987.707576,17792.124399,151.473270,8.338790e+01,9.270841e+01,8.933551e-02,591.239737,656.541220,7.632118
DEE10020,2.006459e-03,7.863594e-05,0.000000e+00,43795.530678,51826.206245,199454.511296,9.970760e-01,9.625948e-01,9.549835e-01,14.552611,186.164073,224.045342
DEE10080,1.293943e-03,3.043924e-03,0.000000e+00,3556.710772,33292.457946,47229.098005,6.249742e+00,3.275233e+02,9.844412e+01,22.842351,1035.105026,313.326664
DEE10120,0.000000e+00,4.584215e-02,0.000000e+00,13373.239145,5071.744990,36895.729757,1.069275e+01,1.938253e+01,4.528384e+01,157.936029,275.310526,625.164181
DEE10370,9.944320e-02,1.640731e-03,0.000000e+00,14934.243119,227822.361401,111270.342611,8.869107e+00,5.927723e+02,1.456964e-01,2077.941260,125018.806745,179.873698
DEE10410,5.413847e-04,7.774266e-01,0.000000e+00,215404.995425,3598.861090,91247.066895,6.391490e+01,4.499625e-02,9.614936e-01,124074.281079,1997.340567,73.598802
DEE10470,9.377393e-04,9.290365e-02,0.000000e+00,4533.169993,3214.306189,139957.495802,4.132329e+00,9.374477e+00,6.364368e-01,69.213411,139.907803,22.068609
DEE10740,2.806567e-03,4.026714e-03,0.000000e+00,12943.914571,16515.575107,127504.616995,9.864783e-01,1.729133e-01,1.070900e+01,2.805672,171.615340,2429.544390
DEB10050,4.509022e-04,6.833367e-03,9.849615e-01,14173.931940,10751.793864,11.566375,1.158070e+02,1.453407e+02,9.301292e-05,986.142399,1235.480892,8.443280


In [12]:
df_results[df_results["area_difference_reported_fed_merit_hydro"] > 0.8]

,overlap_fed_merit_hydro,overlap_fed_hydrosheds,overlap_fed_basis_ezg,centroid_distance_fed_merit_hydro,centroid_distance_fed_hydrosheds,centroid_distance_fed_basis_ezg,area_difference_fed_merit_hydro,area_difference_fed_hydrosheds,area_difference_fed_basis_ezg,area_difference_reported_fed_merit_hydro,area_difference_reported_fed_hydrosheds,area_difference_reported_fed_basis_ezg
DEF10230,0.868923,0.297665,0.999631,255.173907,7923.869169,5.423618,0.048514,2.121373,0.000293,174.345417,519.017328,166.327276
DEF10240,0.892880,0.319983,0.999822,184.046826,8000.597082,3.582561,0.013970,1.948521,0.000177,178.485362,519.017328,176.057543
DEF10580,0.000580,0.084336,0.554219,14346.330182,6793.483150,540.344638,34.348352,9.930871,0.804337,214.559833,66.348947,10.952089
DEF10600,0.842996,0.387489,0.996977,489.144510,4233.097457,33.281897,0.033004,1.210570,0.003031,75.362240,161.270830,73.175522
DEF10670,0.855881,0.148588,0.987052,440.435464,5869.353737,161.253525,0.104257,5.456642,0.013117,88.919575,640.945147,100.571198
...,...,...,...,...,...,...,...,...,...,...,...,...
DE112450,0.876893,0.235952,0.834921,295.173366,3984.839765,742.401819,0.015567,2.981159,0.197717,39.485226,154.787331,46.567203
DE112460,0.561569,0.471507,0.951035,2888.735388,2965.977120,334.949877,0.765662,1.104273,0.051485,168.181215,200.434332,100.155085
DE112480,0.963862,0.807798,0.907852,52.327176,792.374233,852.150620,0.000320,0.206979,0.101285,101.151233,122.126469,111.432015
DE112490,0.843746,0.711506,0.914867,1126.825546,2920.093997,844.191620,0.117942,0.178990,0.050325,318.187411,425.300579,342.579051


In [13]:
import plotly.express as px

s = Station("DE112500")

gdf_federal_agency_ezg = s.get_catchment("federal_agency_ezg")
gdf_merit_hydro = s.get_catchment("merit_hydro")
gdf_hydrosheds = s.get_catchment("hydrosheds")
gdf_basis_ezg = s.get_catchment("basis_ezg")

# plot all on same axis
fig = go.Figure()

fig.add_trace(go.Choroplethmapbox(
    geojson=gdf_federal_agency_ezg.geometry.__geo_interface__,
    z=[1],
    locations=gdf_federal_agency_ezg.index,
    marker_opacity=0.5,
    marker_line_width=0,
    name="Federal Agency EZG",
    colorscale=[[0, "green"], [1, "green"]],
    showlegend=True,
    showscale=False
))

fig.add_trace(go.Choroplethmapbox(
    geojson=gdf_merit_hydro.geometry.__geo_interface__,
    z=[1],
    locations=gdf_merit_hydro.index,
    marker_opacity=0.5,
    marker_line_width=0,
    name="MERIT Hydro",
    colorscale=[[0, "blue"], [1, "blue"]],
    showlegend=True,
    showscale=False
))

fig.add_trace(go.Choroplethmapbox(
    geojson=gdf_hydrosheds.geometry.__geo_interface__,
    z=[1],
    locations=gdf_hydrosheds.index,
    marker_opacity=0.5,
    marker_line_width=0,
    name="HydroSHEDS",
    colorscale=[[0, "red"], [1, "red"]],
    showlegend=True,
    showscale=False
))

fig.add_trace(go.Choroplethmapbox(
    geojson=gdf_basis_ezg.geometry.__geo_interface__,
    z=[1],
    locations=gdf_basis_ezg.index,
    marker_opacity=0.5,
    marker_line_width=0,
    name="BASIS EZG",
    colorscale=[[0, "yellow"], [1, "yellow"]],
    showlegend=True,
    showscale=False
))

fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=9,
    mapbox_center = {"lat": s.lat, "lon": s.lon},
    margin={"r":0,"t":0,"l":0,"b":0},
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

## Now compare MeritHYDRO to HydroSHEDS and Basis EZG

In [9]:
# compare overlap of merit hydro and hydrosheds
# get metadata (only for stations with more than 10 years of Q data)
metadata = get_metadata()
metadata = metadata[metadata["flag_q_more_than_10_years"]]

# get all camels ids
camels_ids = metadata["camels_id"].tolist()

# initialize dictionary to store overlap values
overlap_dict_mh = {}

for camels_id in camels_ids:
    # initialize station
    s = Station(camels_id)

    # get catchments for merit_hydro, hydrosheds and basis_ezg
    gdf_merit_hydro = s.get_catchment("merit_hydro")
    gdf_hydrosheds = s.get_catchment("hydrosheds")
    gdf_basis_ezg = s.get_catchment("basis_ezg")

    if gdf_merit_hydro is None:
        overlap_dict_mh[s.camels_id] = {
            "hydrosheds": None,
            "basis_ezg": None
        }

    else:
        # caluculate percentage of overlap to hydrosheds and basis_ezg
        overlap_hydrosheds = calculate_overlap_percentage_two_catchments(gdf_merit_hydro, gdf_hydrosheds)
        overlap_basis_ezg = calculate_overlap_percentage_two_catchments(gdf_merit_hydro, gdf_basis_ezg)

        overlap_dict_mh[s.camels_id] = {
            "hydrosheds": overlap_hydrosheds,
            "basis_ezg": overlap_basis_ezg
        }

# make dataframe
df_overlap_merit_hydrosheds = pd.DataFrame(overlap_dict_mh).T

/tmp/ipykernel_138927/1097745454.py:23: UserWarning:

`keep_geom_type=True` in overlay resulted in 50 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries



In [36]:
df_overlap_mh = pd.DataFrame(overlap_dict_mh, index=["overlap"]).T

# plot histogram with plotly go
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_overlap_mh["overlap"], xbins=dict(start=0, end=1, size=0.05)))
fig.update_layout(barmode='overlay', title="Overlap of MERIT Hydro Catchments with HydroSHEDS")
fig.update_traces(opacity=0.6)
fig.update_xaxes(title_text="Overlap [%]")
fig.update_yaxes(title_text="Count")
fig.show()

## Compare area_calc with area_reported

We can also compare the area from the metadata (provided by federal agencies) with the calculated area of the merit hydro catchments.

In [11]:
# GeoDataFrame of all catchments
camels_ids = get_metadata()["camels_id"].values

# create a list of geodataframes
gdfs = []

for camels_id in camels_ids:
    # initialize station
    s = Station(camels_id)

    # get merit hydro catchment
    gdf = s.get_catchment("merit_hydro")

    if isinstance(gdf, gpd.GeoDataFrame):
        gdfs.append(gdf)

# concat all geodataframes
gdf_merit_all = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

gdf_merit_all

/tmp/ipykernel_138927/1557730114.py:18: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,id,name,result,area_calc,area_reported,geometry
0,DEG10000,Ammern,High Res,184.0,182.700,"POLYGON ((10.44542 51.23125, 10.44458 51.23125..."
1,DEG10010,Arenshausen,High Res,273.0,275.000,"POLYGON ((10.06708 51.41458, 10.07375 51.41458..."
2,DEG10020,Arnstadt,High Res,175.0,174.700,"POLYGON ((10.81792 50.77208, 10.81792 50.77292..."
3,DEG10030,Berga,High Res,1390.0,1383.000,"POLYGON ((12.15625 50.75042, 12.15625 50.75042..."
4,DEG10040,Blankenstein-Rosenthal,High Res,766.0,1013.000,"POLYGON ((11.78458 50.37292, 11.77375 50.37292..."
...,...,...,...,...,...,...
2819,DE112470,Blaubeuren,High Res,171.0,0.067,"POLYGON ((9.55958 48.45125, 9.55958 48.45208, ..."
2820,DE112480,Erlenbach,High Res,101.0,101.510,"POLYGON ((9.27875 49.20625, 9.27875 49.20542, ..."
2821,DE112490,Bolheim,High Res,318.0,339.811,"POLYGON ((10.23958 48.67792, 10.23958 48.67458..."
2822,DE112500,Schweinhausen,High Res,85.5,101.589,"POLYGON ((9.74208 48.05792, 9.74208 48.05708, ..."
